<a href="https://colab.research.google.com/github/Milafreire/projeto_extract_datasus_to_bigquery_with_mongodb_pyspark/blob/main/criacao_datawarehouse_datasus_transform_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymongo
!pip install pyspark
!pip install spark
!pip install google.cloud bigquery google.cloud storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=32b7136dc6f50d22c159e6c872c108f233ed8210f6c3d02352ceac6abf7f139d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58749 sha256=5cc96ffc2464f7c3d78f505b35a5da3890fa557351ef4e60a78703126a605f67
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark
  Preparing metad

In [31]:
#Credentials aqui

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, to_date
from google.cloud.exceptions import NotFound
from pymongo import MongoClient
import os
import pandas as pd
from google.cloud import bigquery
from google.cloud import storage

In [30]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

os.environ['Google_cloud_projects'] = f'{project_id}'

Authenticated


In [33]:
#Iniciando as conexões mongodb e spark
def get_mongo_client(uri, database_name, collection_name):
    client = MongoClient(uri)
    db = client[database_name]
    return db[collection_name]

def create_spark_session(app_name="MongoDBToPostgres"):
    return SparkSession.builder \
        .appName(app_name) \
        .master("local[*]") \
        .getOrCreate()

# Função para ler dados do MongoDB
def read_data_from_mongo(collection):
    return list(collection.find())

# Função para criar DataFrame no Spark
def create_spark_dataframe(spark, data):
    return spark.createDataFrame(data)

In [34]:
#Função para processamento dos dados
def process_data(df):
    df_exploded = df.select(
        col("_id"),
        col("_index"),
        col("_score"),
        col("_source.paciente_endereco_coIbgeMunicipio").alias("paciente_endereco_coIbgeMunicipio"),
        col("_source.vacina_fabricante_referencia").alias("vacina_fabricante_referencia"),
        col("_source.vacina_descricao_dose").alias("vacina_descricao_dose"),
        col("_source.vacina_lote").alias("vacina_lote"),
        col("_source.vacina_fabricante_nome").alias("vacina_fabricante_nome"),
        col("_source.estabelecimento_razaoSocial").alias("razaoSocial"),
        col("_source.id_sistema_origem").alias("id_sistema_origem"),
        col("_source.document_id").alias("document_id"),
        col("_source.vacina_codigo").alias("vacina_codigo"),
        col("_source.estabelecimento_valor").alias("estabelecimento_cod"),
        to_date(col("_source.data_importacao_rnds")).alias("importacao_rnds"),
        col("_source.vacina_grupoAtendimento_codigo").alias("atendimento_cod"),
        col("_source.@timestamp").alias("@timestamp"),
        col("_source.dt_deleted").alias("dt_deleted"),
        col("_source.paciente_endereco_nmMunicipio").alias("paciente_cidade"),
        col("_source.vacina_numDose").alias("v_numDose"),
        col("_source.paciente_endereco_uf").alias("paciente_uf"),
        col("_source.paciente_racaCor_valor").alias("paciente_raca"),
        col("_source.co_condicao_maternal").alias("co_condicao_maternal"),
        col("_source.data_importacao_datalake").alias("data_importacao_datalake"),
        col("_source.paciente_id").alias("paciente_id"),
        col("_source.paciente_enumSexoBiologico").alias("paciente_sexo"),
        col("_source.@version").alias("@version"),
        col("_source.estabelecimento_municipio_nome").alias("estabelecimento_municipio_nome"),
        col("_source.estabelecimento_uf").alias("estabelecimento_uf"),
        col("_source.paciente_endereco_nmPais").alias("paciente_enderecos"),
        col("_source.paciente_endereco_cep").alias("paciente_cep"),
        col("_source.vacina_categoria_nome").alias("vacina_categ"),
        col("_source.status").alias("status"),
        col("_source.paciente_racaCor_codigo").alias("paciente_raca_cod"),
        col("_source.sistema_origem").alias("sistema_origem"),
        col("_source.paciente_endereco_coPais").alias("pais"),
        col("_source.paciente_nacionalidade_enumNacionalidade").alias("nacionalidade"),
        col("_source.vacina_grupoAtendimento_nome").alias("v_Atendimento_nome"),
        to_date(col("_source.vacina_dataAplicacao")).alias("v_dataAplicacao"),
        col("_source.vacina_nome").alias("vacina_nome"),
        col("_source.ds_condicao_maternal").alias("ds_condicao_maternal"),
        col("_source.vacina_categoria_codigo").alias("vacina_categoria_codigo"),
        col("_source.paciente_idade").alias("paciente_idade"),
        col("_source.estabelecimento_municipio_codigo").alias("estabelecimento_municipio_codigo"),
        col("_source.estalecimento_noFantasia").alias("estalecimento"),
        col("_source.paciente_dataNascimento").alias("paciente_nascimento")
    )

    df_cleaned = df_exploded.filter(df_exploded.paciente_id.isNotNull()).fillna('')
    df_cleaned = df_cleaned.drop("_id", "_index", "_score", 'dt_deleted', 'atendimento_cod', 'paciente_endereco_coPais', 'status', "paciente_endereco_coIbgeMunicipio", "vacina_fabricante_referencia", 'vacina_categoria_codigo', 'data_importacao_datalake', 'ds_condicao_maternal', '@version', 'data_importacao_datalake', 'co_condicao_maternal', '@timestamp')

    df_cleaned.createOrReplaceTempView("datasus_data")

    return df_cleaned

In [21]:
# Executar Queries para criação das colunas estado_nome e faixa-etaria
def execute_queries(spark, df_cleaned):
    result_df1 = spark.sql(
        """
        SELECT *,
        CASE
            WHEN CAST(paciente_idade AS INT) >= 0 AND CAST(paciente_idade AS INT) <= 9 THEN '0-9'
            WHEN CAST(paciente_idade AS INT) >= 10 AND CAST(paciente_idade AS INT) <= 19 THEN '10-19'
            WHEN CAST(paciente_idade AS INT) >= 20 AND CAST(paciente_idade AS INT) <= 29 THEN '20-29'
            WHEN CAST(paciente_idade AS INT) >= 30 AND CAST(paciente_idade AS INT) <= 39 THEN '30-39'
            WHEN CAST(paciente_idade AS INT) >= 40 AND CAST(paciente_idade AS INT) <= 49 THEN '40-49'
            WHEN CAST(paciente_idade AS INT) >= 50 AND CAST(paciente_idade AS INT) <= 59 THEN '50-59'
            WHEN CAST(paciente_idade AS INT) >= 60 AND CAST(paciente_idade AS INT) <= 69 THEN '60-69'
            WHEN CAST(paciente_idade AS INT) >= 70 AND CAST(paciente_idade AS INT) <= 79 THEN '70-79'
            WHEN CAST(paciente_idade AS INT) >= 80 AND CAST(paciente_idade AS INT) <= 89 THEN '80-89'
            WHEN CAST(paciente_idade AS INT) >= 90 AND CAST(paciente_idade AS INT) <= 99 THEN '90-99'
            ELSE '100+'
        END AS faixa_etaria
        FROM datasus_data
        """
    )

    result_df2 = spark.sql(
        """
        SELECT *,
        CASE
            WHEN estabelecimento_uf = 'AC' THEN 'Acre'
            WHEN estabelecimento_uf = 'AL' THEN 'Alagoas'
            WHEN estabelecimento_uf = 'AP' THEN 'Amapá'
            WHEN estabelecimento_uf = 'AM' THEN 'Amazonas'
            WHEN estabelecimento_uf = 'BA' THEN 'Bahia'
            WHEN estabelecimento_uf = 'CE' THEN 'Ceará'
            WHEN estabelecimento_uf = 'DF' THEN 'Distrito Federal'
            WHEN estabelecimento_uf = 'ES' THEN 'Espírito Santo'
            WHEN estabelecimento_uf = 'GO' THEN 'Goiás'
            WHEN estabelecimento_uf = 'MA' THEN 'Maranhão'
            WHEN estabelecimento_uf = 'MT' THEN 'Mato Grosso'
            WHEN estabelecimento_uf = 'MS' THEN 'Mato Grosso do Sul'
            WHEN estabelecimento_uf = 'MG' THEN 'Minas Gerais'
            WHEN estabelecimento_uf = 'PA' THEN 'Pará'
            WHEN estabelecimento_uf = 'PB' THEN 'Paraíba'
            WHEN estabelecimento_uf = 'PR' THEN 'Paraná'
            WHEN estabelecimento_uf = 'PE' THEN 'Pernambuco'
            WHEN estabelecimento_uf = 'PI' THEN 'Piauí'
            WHEN estabelecimento_uf = 'RJ' THEN 'Rio de Janeiro'
            WHEN estabelecimento_uf = 'RN' THEN 'Rio Grande do Norte'
            WHEN estabelecimento_uf = 'RS' THEN 'Rio Grande do Sul'
            WHEN estabelecimento_uf = 'RO' THEN 'Rondônia'
            WHEN estabelecimento_uf = 'RR' THEN 'Roraima'
            WHEN estabelecimento_uf = 'SC' THEN 'Santa Catarina'
            WHEN estabelecimento_uf = 'SP' THEN 'São Paulo'
            WHEN estabelecimento_uf = 'SE' THEN 'Sergipe'
            WHEN estabelecimento_uf = 'TO' THEN 'Tocantins'
            ELSE 'Unknown'
        END AS estado_nome
        FROM datasus_data
        """
    )
    # Renomear colunas nos DataFrames resultantes
    result_df1 = result_df1.selectExpr(
        "paciente_id as paciente_id_1",
        "paciente_sexo as paciente_sexo_1",
        "paciente_raca as paciente_raca_1",
        "paciente_cidade as paciente_cidade_1",
        "nacionalidade as nacionalidade_1",
        "paciente_idade as paciente_idade_1",
        "faixa_etaria as faixa_etaria_1"
    )

    result_df2 = result_df2.selectExpr(
        "paciente_id as paciente_id_2",
        "razaoSocial as razaoSocial_2",
        "estabelecimento_municipio_nome as estabelecimento_municipio_nome_2",
        "estabelecimento_uf as estabelecimento_uf_2",
        "estalecimento as estalecimento_2",
        "estado_nome as estado_nome_2",
        "vacina_descricao_dose as vacina_descricao_dose_2",
        "vacina_fabricante_nome as vacina_fabricante_nome_2",
        "vacina_codigo as vacina_codigo_2",
        "estabelecimento_cod as estabelecimento_cod_2",
        "v_numDose as v_numDose_2",
        "vacina_categ as vacina_categ_2",
        "v_dataAplicacao as v_dataAplicacao_2",
        "vacina_nome as vacina_nome_2"
    )

    #Join para unir dos dfs resultantes
    combined_df = result_df1.join(result_df2, result_df1['paciente_id_1'] == result_df2['paciente_id_2'])

    # Selecionar colunas finais do DataFrame combinado
    final_df = combined_df.select(
      col('paciente_id_1').alias('paciente_id'),
      col('paciente_sexo_1').alias('paciente_sexo'),
      col('paciente_raca_1').alias('paciente_raca'),
      col('paciente_cidade_1').alias('paciente_cidade'),
      col('nacionalidade_1').alias('nacionalidade'),
      col('paciente_idade_1').alias('paciente_idade'),
      col('faixa_etaria_1').alias('faixa_etaria'),
      col('razaoSocial_2').alias('razaoSocial'),
      col('estabelecimento_municipio_nome_2').alias('estabelecimento_municipio_nome'),
      col('estabelecimento_uf_2').alias('estabelecimento_uf'),
      col('estalecimento_2').alias('estalecimento'),
      col('estado_nome_2').alias('estado_nome'),
      col('vacina_descricao_dose_2').alias('vacina_descricao_dose'),
      col('vacina_fabricante_nome_2').alias('vacina_fabricante_nome'),
      col('vacina_codigo_2').alias('vacina_codigo'),
      col('estabelecimento_cod_2').alias('estabelecimento_cod'),
      col('v_numDose_2').alias('v_numDose'),
      col('vacina_categ_2').alias('vacina_categ'),
      col('v_dataAplicacao_2').alias('v_dataAplicacao'),
      col('vacina_nome_2').alias('vacina_nome')
  )

    # Converter DataFrame final para Pandas
    final_pandas_df = final_df.toPandas()

In [25]:
#Schemas
schema_dim_paciente = [
    bigquery.SchemaField("paciente_id", "STRING"),
    bigquery.SchemaField("paciente_sexo", "STRING"),
    bigquery.SchemaField("paciente_raca", "STRING"),
    bigquery.SchemaField("paciente_cidade", "STRING"),
    bigquery.SchemaField("paciente_nacionalidade", "STRING"),
    bigquery.SchemaField("paciente_idade", "STRING"),
    bigquery.SchemaField("faixa_etaria", "STRING")
]

schema_dim_localizacao = [
    bigquery.SchemaField("paciente_id", "STRING"),
    bigquery.SchemaField("razaoSocial", "STRING"),
    bigquery.SchemaField("estabelecimento_municipio_nome", "STRING"),
    bigquery.SchemaField("estabelecimento_uf", "STRING"),
    bigquery.SchemaField("estalecimento", "STRING"),
    bigquery.SchemaField("estado_nome", "STRING")
]

schema_fato_atendimento = [
    bigquery.SchemaField("vacina_descricao_dose", "STRING"),
    bigquery.SchemaField("vacina_fabricante_nome", "STRING"),
    bigquery.SchemaField("paciente_id", "STRING"),
    bigquery.SchemaField("vacina_codigo", "STRING"),
    bigquery.SchemaField("estabelecimento_cod", "STRING"),
    bigquery.SchemaField("v_numDose", "STRING"),
    bigquery.SchemaField("vacina_categ", "STRING"),
    bigquery.SchemaField("v_dataAplicacao", "DATE"),
    bigquery.SchemaField("vacina_nome", "STRING")
]

In [40]:
# Função para criar tabela se não existir
def create_table_if_not_exists(client, table_id, schema):
    try:
        client.get_table(table_id)
        print(f"Table {table_id} already exists.")
    except NotFound:
        table = bigquery.Table(table_id, schema=schema)
        client.create_table(table)
        print(f"Table {table_id} created.")

In [37]:
# Inicializar cliente BigQuery
client = bigquery.Client(project=project_id)

In [38]:
#Função para inserção dos dados no Big query
def load_parquet_to_bq(client, uri, table_id, schema):
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        source_format=bigquery.SourceFormat.PARQUET,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
    )
    load_job = client.load_table_from_uri(uri, table_id, job_config=job_config)
    load_job.result()
    destination_table = client.get_table(table_id)
    print(f"Loaded {destination_table.num_rows} rows to {table_id}")

In [29]:
# IDs das tabelas
table_id_dim_paciente = f"{project_id}.{dataset_id}.dim_paciente"
table_id_dim_localizacao = f"{project_id}.{dataset_id}.dim_localizacao"
table_id_fato_atendimento = f"{project_id}.{dataset_id}.fato_atendimento"

# URIs dos arquivos Parquet no GCS
gcs_uri_dim_paciente = f"gs://{pasta_storage}/parquet_files/dim_paciente.parquet"
gcs_uri_dim_localizacao = f"gs://{pasta_storage}/parquet_files/dim_localizacao.parquet"
gcs_uri_fato_atendimento = f"gs://{pasta_storage}/parquet_files/fato_atendimento.parquet"

# Criar tabelas no BigQuery se não existirem
create_table_if_not_exists(client, table_id_dim_paciente, schema_dim_paciente)
create_table_if_not_exists(client, table_id_dim_localizacao, schema_dim_localizacao)
create_table_if_not_exists(client, table_id_fato_atendimento, schema_fato_atendimento)

# Carregar arquivos Parquet no BigQuery
load_parquet_to_bq(client, gcs_uri_dim_paciente, table_id_dim_paciente, schema_dim_paciente)
load_parquet_to_bq(client, gcs_uri_dim_localizacao, table_id_dim_localizacao, schema_dim_localizacao)
load_parquet_to_bq(client, gcs_uri_fato_atendimento, table_id_fato_atendimento, schema_fato_atendimento)

Table mywebscrap-423316.estudos_gcp.dim_paciente already exists.
Table mywebscrap-423316.estudos_gcp.dim_localizacao already exists.
Table mywebscrap-423316.estudos_gcp.fato_atendimento already exists.
Loaded 14544 rows to mywebscrap-423316.estudos_gcp.dim_paciente
Loaded 14544 rows to mywebscrap-423316.estudos_gcp.dim_localizacao
Loaded 14544 rows to mywebscrap-423316.estudos_gcp.fato_atendimento
